In [1]:
from lxml import etree
# read in raw file 
Dataset = "DBLP"
filename = "../Data/"+Dataset+"/dblp.xml"

In [ ]:
import io

def reset():
    author_array = []
    title = ""
    year = ""
    journal = ""
    return author_array, title, year, journal

collaborations = ['www', 'phdthesis', 'inproceedings', 'incollection', 'proceedings', 'book', 'mastersthesis', 'article']
count_vaild_record = 0

author_array, title, year, journal = reset()
have_orcid_author = []
author_orcid = []
# iterparse iter through tag element in file and build up the tree
nodes = etree.iterparse(filename, html=True)
for event, elem in nodes:
    # we only extract useful information for our experiment
    if elem.tag == "author":
        author_array.append(elem.text)
        if 'orcid' in elem.attrib:
            have_orcid_author.append(elem.text)
            author_orcid.append(elem.attrib['orcid'])
    if elem.tag == "title":
        title = elem.text
    if elem.tag == "year":
        year = elem.text
    if elem.tag == "journal":
        journal = elem.text
    
    if elem.tag in collaborations:
        # no author name or no title will be removed
        if len(author_array)== 0 or title == None:
            # reset article information
            author_array, title, year, journal = reset()
            have_orcid_author = []
            author_orcid = []
        else:
            #print(author_array)
            # only save papers have orcid
            if len(have_orcid_author)>0:
                # print(len(have_orcid_author))
                for orcid, author in zip(author_orcid, have_orcid_author):
                    count_vaild_record+=1
                    main_author = str(author)
                    main_orcid = orcid
                    # concate string as one line
                    co_author = author_array[:]
                    co_author.remove(author)
                    # if only one author write this paper and he have orcid, co_author string will be empty
                    if len(co_author)== 0:
                        co_author_string = ""
                    else:
                        co_author_string = "$$".join(co_author)
                    out_sample = main_orcid+"\t"+main_author+"\t"+co_author_string+"\t"+title+"\t"+year+"\t"+journal
                    # print(out_sample)
                    # append to file
                    with io.open("../Data/DBLP/extracted_labeled_data.txt", "a", encoding='utf8') as myfile:
                        myfile.write(out_sample+"\n")
                # reset article information
                have_orcid_author = []
                author_orcid = []
                author_array, title, year, journal = reset()
            else:
                # reset article information
                author_array, title, year, journal = reset()
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]
del nodes
print(count_vaild_record)